In [ ]:
import re
import csv
import datetime
import requests
import json
import base64
import duckdb
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt

API_KEY_REF = "YOUR_API_KEY_HERE" # place your API key here
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {API_KEY_REF}",
    "Content-Type": "application/json"
}
PDF_PATH = "ILEC Mortality Report.pdf"
DATA_DICTIONARY_PATH = "ILEC 2012_19 - Data Dictionary clean.csv"
DATA_SAMPLE_PATH = "ILEC_Sample.csv"

def encode_to_base64(filepath):
    """
    Description:
        Encode an arbitrary file (e.g. PDF, image) so that it can be passed as input to an LLM through a POST request

    Arguments:
        filepath (str): path to the file to be encoded
    """
    with open(filepath, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def make_prompt_with_data_dict_and_sample(prompt, data_dict_path, sample_csv_path=None):
    """
    Description:
        Return a string starting with a text representation of the provided data dictionary and sample csv file, along with user-provided prompt. Used for convenience across several types of LLM requests

    Arguments:
        prompt (str): prompt that will appear at the end of the user's request
        data_dict_path (str): path to a csv file containing the data dictionary
        sample_csv_path (str): path to a csv file containing a handful of records from the dataset
    """
    output = "ILEC_Mortality_Table Data Dictionary:\n\n"
    with open(data_dict_path, "r") as f:
        output += "".join(f.readlines())
    if sample_csv_path is not None:
        output += "\n\nSample data from ILEC_Mortality_Table:\n\n"
        with open(sample_csv_path, "r") as f:
            for i in range(5): # load up to the first five lines of the file
                new_line = f.readline()
                if new_line:
                    output += new_line
                else:
                    break
    return output + "\nTask:\n" + prompt

def get_initial_ideas(model_name, structure_path, output_file=None):
    """
    Description:
        Request a list of ideas from a model based on report PDF and data. Log the results.
    Note: this function contains some hardcoded details (PDF file for retrieval)

    Arguments:
        model_name (str): specify the model to be used
        structure_path (str): path to JSON file with the desired "structure output" that the LLM should follow
        output_file (str): path to save output of LLM response (JSON)
    """
    prompt = """I've provided an insurance mortality report with findings based on 2012 to 2019 data. I've also provided a data dictionary and a small data extract. Based on the report, identify several new ideas for how we can extend the analysis using only the data provided (e.g. additional cuts/segmentations). In your description, tie in your extension idea to the paper (e.g. your inspiration) and make sure that your suggested view is not already provided in the paper. You should list relevant dimensions of the cut. You MUST use observation_year, since we want to detect shifting trends over time. If the idea requires any filtering, please justify that choice in the response as well. We are only interested in actual vs expected trends (specifically, ratio of actual amount to expected amount) or mix shifts across the observation years (2012 through 2019) for segments to see how patterns are changing over time. The test should be designed so that we can identify trends based on a single graph. Note that if there are too many unique values for a dimension, we would not easily be able to compare trends, so in cases of 6 or more unique values (per the data dictionary), we must either filter to a smaller list or group values together. For each of your extension ideas, propose a hypothesis and what new trend insights we expect to learn from your proposed view. Try to come up with at least 4 ideas."""

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": make_prompt_with_data_dict_and_sample(prompt, DATA_DICTIONARY_PATH, DATA_SAMPLE_PATH)
                },
                {
                    "type": "file",
                    "file": {
                        "filename": PDF_PATH,
                        "file_data": f"data:application/pdf;base64,{encode_to_base64(PDF_PATH)}"
                    }
                },
            ]
        }
    ]
    plugins = [{"id": "file-parser","pdf": {"engine": "pdf-text"}}]
    payload = {
        "model": model_name,
        "messages": messages,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "idea_list",
                "strict": True,
                "schema": json.load(open(structure_path, "r"))
            }
        },
        "require_parameters": True,
        "plugins": plugins
    }

    try:
        response = requests.post(OPENROUTER_URL, headers=HEADERS, json=payload)
    except Exception as e:
        print(f"Error during API request: {e}")
    if output_file is None:
        curr_timestamp = datetime.datetime.now().isoformat()[:-7]
        output_file = f"ideas_response_{curr_timestamp.replace(':', '')}.json"
    with open(output_file, "w") as f:
        json.dump(response.json(), f, indent=4)
    print(f"Successfully evaluated initial ideas!")
    return response.json()

def get_sql_code(model_name, query_request, query_structure_path, save_query_file):
    """
    Description:
        Make a request to an LLM to generate a SQL query based on the provided information, conforming to the template specified

    Arguments:
        model_name (str): specify the model to be used
        query_request (dict):cContains details of the goal/purpose to help the LLM generate an appropriate SQL query
        query_structure_path (str): path to a JSON file with a specified structured output
        save_query_file (str): path to a file indicating where to save the provided query
    """
    prompt = f"""Write a SQL query based on the following overview in order to test the hypothesis. The general intent is to identify actual vs expected trends (actual/ratio is supposed to be a ratio) or mix shifts across the observation years (2012 to 2019) for segments to see how patterns are changing over time. The table to use is called ILEC_Mortality_Table. If the query creates an actual vs expected column, it should be called AE_Ratio_Amount. If the query creates a mix percentage column, it should be called Mix_Percentage. Pay attention to the datatypes listed in the data dictionary - avoid using CAST to change datatypes. Additionally, identify all the segment columns (e.g. Observation_Year) for which we'll look at trends. You should reason through how the query answers the request mentioned in the Request and Hypothesis section.\n\nRequest:\n{query_request['overview']}\n\nHypothesis:\n{query_request['hypothesis']}\n\nPossible Fields:\n{query_request['dimensions']}""" + (f"\n\nPossible Filters:\n{query_request['filters']}" if "filters" in query_request.keys() else "")
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": make_prompt_with_data_dict_and_sample(prompt, DATA_DICTIONARY_PATH, DATA_SAMPLE_PATH)
                }
            ]
        }
    ]
    payload = {
        "model": model_name,
        "messages": messages,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "sql_info",
                "strict": True,
                "schema": json.load(open(query_structure_path, "r"))
            }
        },
        "require_parameters": True
    }
    try:
        response = requests.post(OPENROUTER_URL, headers=HEADERS, json=payload)
    except Exception as e:
        print(f"Error during API request: {e}")
    with open(save_query_file, "w") as f:
        json.dump(response.json(), f, indent=4)
    print(f"Successfully evaluated!")
    return response.json()

def get_all_sql_code(model_name, idea_response_file, query_structure_path, output_file):
    """
    Description:
        Send a request to specified LLM to generate a SQL query for each idea passed in

    Arguments:
        model_name (str): specify the model to be used
        idea_response_file (str): path to JSON file containing LLM's ideas
        query_structure_path (str): path to a JSON file with a specified structured output
        output_file (str): path to JSON file indicating where to save the query's full details
    """
    output_data = []
    with open(idea_response_file, "r") as f:
        idea_response = json.load(f)
        ideas = json.loads(idea_response["choices"][0]["message"]["content"])["ideas"]
    for i, idea_obj in enumerate(ideas):
        print(f"Processing {i+1} out of {len(ideas)}")
        curr_timestamp = datetime.datetime.now().isoformat()[:-7]
        random_file_name = f"response_{curr_timestamp.replace(':', '')}.json"
        res = get_sql_code(model_name, idea_obj, query_structure_path, random_file_name)
        sql_content = json.loads(res["choices"][0]["message"]["content"])
        output_data.append({
            "json_request_location": random_file_name,
            "overview": idea_obj["overview"],
            "hypothesis": idea_obj["hypothesis"],
            "dimensions": idea_obj["dimensions"],
            "filters": idea_obj.get("filters", ""),
            "group_column_list": sql_content["group_column_list"],
            "sql_desc": sql_content["description"],
            "sql_code": sql_content["sql_code"]
        })
    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=4)
    #print('Done!')
    return output_data

def exec_query(sql_query, save_location):
    """
    Description:
        Clean up the provided SQL query and save it to the specified location

    Arguments:
        sql_query (str): SQL query, created by an LLM
        save_location (str): path to a CSV file where the output data should be saved
    """
    sql_query = sql_query.strip()
    if not sql_query.lower().startswith("select") and not sql_query.lower().startswith("with"):
        #print(sql_query)
        raise ValueError("Malformed query: query does not start with 'select' or 'with'")
    sql_query = sql_query.replace('"', "'") # replace with single quotes because duckdb is weird
    if re.search(r'\silec_mortality_table(\s|;)', sql_query, flags=re.IGNORECASE):
        sql_query = re.sub(r'ilec_mortality_table', "read_csv_auto('ILEC_2012_19 - 20240429.txt', delim='\t', types={'Age_Ind': 'VARCHAR', 'Preferred_Indicator': 'VARCHAR', 'Number_of_Pfd_Classes': 'VARCHAR', 'Preferred_Class': 'VARCHAR'})", sql_query, flags=re.IGNORECASE)
    else:
        raise ValueError("Malformed query: could not find 'ilec_mortality_table' in query")
    if sql_query.endswith(";"):
        sql_query = sql_query[:-1]
    duckdb.sql(f"""COPY ({sql_query}) TO '{save_location}';""")

def make_graph(data_path, query_detail):
    """
    Description:
        Plots the data provided and saves it to the specified location

    Arguments:
        data_path (str): path to where the csv dataset is saved
        query_info (dict): JSON object containing supplemental query information as additional context
    """
    df = pd.read_csv(data_path)
    df.columns = [col.lower() for col in df.columns]
    
    # Identify segment columns (everything that isn't the year or a numeric metric)
    metrics = ["total_actual_death_amount", "total_expected_death_amount", "ae_ratio_amount", "mix_percentage"]
    # try to figure out what the primary metric is:
    if "ae_ratio_amount" in df.columns:
        final_metric_col = "ae_ratio_amount"
    elif "mix_percentage" in df.columns:
        final_metric_col = "mix_percentage"
    else:
        final_metric_col = df.columns[-1]
    #segment_cols = [c for c in df.columns if c != "observation_year" and c not in metrics and c != final_metric_col]
    segment_cols = [col.lower() for col in query_detail["group_column_list"] if col.lower() != "observation_year"]

    plt.figure(figsize=(10, 6))

    # Group by the list of segment columns
    for keys, group in df.groupby(segment_cols):
        # Create a label by joining segment values (handles one or multiple segments)
        label = " | ".join(map(str, keys)) if isinstance(keys, tuple) else str(keys)
        
        # Sort by year to ensure lines connect correctly
        group = group.sort_values("observation_year")
        plt.plot(group["observation_year"], group[final_metric_col], marker="o", label=label)

    plt.xlabel("Observation Year")
    plt.ylabel(final_metric_col)
    plt.title(f"{final_metric_col} by ({segment_cols})")
    plt.legend(title="Segments", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.savefig(query_detail["graph_data_path"])
    #plt.show()

def query_data(all_queries_path, output_path):
    """
    Description:
        Execute each query in provided JSON file and make a line graph based on the query's csv output. Save results in another JSON object

    Arguments:
        all_queries_path (str): path to JSON file with information regarding all queries
        output_path (str): path to JSON file with additional information for all given queries
    """
    with open(all_queries_path, "r") as f:
        all_queries = json.load(f)
    save_results = []
    for i, query_obj in enumerate(all_queries):
        print(f"Executing query #{i+1}")
        curr_timestamp = datetime.datetime.now().isoformat()[:-7]
        random_file_name = f"agg_data_{curr_timestamp.replace(':', '')}.csv"
        query_obj["csv_data_path"] = random_file_name
        query_obj["graph_data_path"] = f"graph_{curr_timestamp.replace(':', '')}.jpeg"
        try:
            exec_query(query_obj["sql_code"], random_file_name)
            query_obj["calc_status"] = "Success"
        except Exception as e:
            query_obj["calc_status"] = f"Failure: {e}"
            query_obj["graph_status"] = f"Failure: cannot make graph due to failed csv creation"
        if query_obj["calc_status"] == "Success":
            try:
                #make_graph(random_file_name, query_obj["graph_data_path"])
                make_graph(random_file_name, query_obj)
                query_obj["graph_status"] = "Success"
            except Exception as e:
                query_obj["graph_status"] = f"Failure: {e}"
        save_results.append(query_obj)
    with open(output_path, "w") as f:
        json.dump(save_results, f, indent=4)
    return save_results

def get_llm_graph_insights(model_name, query_details, output_location):
    """
    Description:
        Get insight from LLM on given query + graph and save the response

    Arguments:
        model_name (str): specify the model to be used
        query_info (dict): JSON object containing supplemental query information as additional context
        output_location (str): path to JSON file where the LLM response should be saved
    """
    prompt = f"""Overview:\n{query_details['overview']}\n\nHypothesis:\n{query_details['hypothesis']}\n\nRequest:\nThe attached graph shows how life insurance mortality actual vs expected ratios (based on amount) are trending from 2012 to 2019 across various segments. I've also provided a data dictionary to help interpret the meaning of the segment and the original rationale for performing the test. Based on the graph and provided context, identify several new insights regarding the presence of a trend (or lack of a trend) and speculate on reasons why we might be seeing a higher or lower than expected mortality for various segments. Also comment on the hypothesis presented."""

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": make_prompt_with_data_dict_and_sample(prompt, DATA_DICTIONARY_PATH)
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_to_base64(query_details['graph_data_path'])}"
                    }
                },
            ]
        }
    ]
    payload = {
        "model": model_name,
        "messages": messages
    }

    try:
        response = requests.post(OPENROUTER_URL, headers=HEADERS, json=payload)
    except Exception as e:
        print(f"Error during API request when generating LLM's insights on graph: {e}")
        return None
    with open(output_location, "w") as f:
        json.dump(response.json(), f, indent=4)
    return response.json()

def get_all_interpretations(model_name, result_file, output_file):
    """
    Description:
        Get all insight from LLM on graphs and saves results
    Arguments:
        result_file (str): path to JSON file with query information + graph location
        output_file (str): path to JSON file to save query information + LLM insights
    """
    with open(result_file, "r") as f:
        all_query_details = json.load(f)
    final_results = []
    for query_detail in all_query_details:
        if query_detail["graph_status"] == "Success":
            curr_timestamp = datetime.datetime.now().isoformat()[:-7]
            random_file_name = f"insight_{curr_timestamp.replace(':', '')}.json"
            insight_obj = get_llm_graph_insights(model_name, query_detail, random_file_name)
        else:
            insight_obj = None
        if insight_obj is None:
            query_detail["insight_status"] = "Failure"
        else:
            query_detail["insight_status"] = "Success"
            query_detail["insight_file"] = random_file_name
            query_detail["insight"] = insight_obj["choices"][0]["message"]["content"]
        final_results.append(query_detail)
    with open(output_file, "w") as f:
        json.dump(final_results, f, indent=4)
    return final_results

def create_markdown_files(full_details_path, file_prefix="idea"):
    """
    Description:
        Create a markdown file for each "idea"

    Arguments:
        full_details_path (str): path to JSON file with query + results information
        file_prefix (str): prefix for each markdown file generated
    """
    with open(full_details_path, "r") as f:
        all_query_details = json.load(f)
    for i, query_detail in enumerate(all_query_details):
        output_text = "# "
        output_text += query_detail["overview"] + "\n\n"
        output_text += "### Hypothesis\n" + query_detail["hypothesis"] + "\n\n"
        output_text += "### Graph" + "\n" + f"![Plot of trend]({query_detail['graph_data_path']})\n\n"
        output_text += "### Insight\n" + query_detail["insight"] + "\n\n"
        output_text += "### SQL Code:\n"
        output_text += "```" + query_detail["sql_code"] + "```"
        with open(f"{file_prefix}_{i+1}.md", "w") as f:
            f.write(output_text)
    return output_text

res = get_initial_ideas("google/gemini-3-flash-preview", "expected_ideas_structure.json", "ideas_log.json")
res2 = get_all_sql_code("google/gemini-3-flash-preview", "ideas_log.json", "sql_structured_output_format.json", "all_sql_queries.json")
res3 = query_data("all_sql_queries.json", "sql_queries_with_results.json")
res4 = get_all_interpretations("google/gemini-3-flash-preview", "sql_queries_with_results.json", "llm_insights.json")
res5 = create_markdown_files("llm_insights.json", "idea")
print("Done!")